# Chroma
`pip install langchain-chroma`
Chroma是一个 AI 原生开源矢量数据库，专注于提高开发人员的工作效率和幸福感。Chroma 已获得 Apache 2.0 许可。

Chroma 以多种模式运行。请参阅下文中与 LangChain 集成的示例。
- `in-memory`- 在 Python 脚本或 Jupyter Notebook 中
- `in-memory with persistance`- 在脚本或笔记本中保存/加载到磁盘
- `in a docker container`- 作为在本地机器或云端运行的服务器

与任何其他数据库一样，您可以：
- .add
- .get
- .update
- .upsert
- .delete
- .peek
- 并.query运行相似性搜索。       
查看完整文档：[docs](https://docs.trychroma.com/)。要直接访问这些方法，您可以这样做._collection.method()

<img src="https://docs.trychroma.com/img/hrm4.svg">

## chromadb 简单尝试


In [4]:
###  pip install chromadb
## 创建 Chroma 客户端
import chromadb
chroma_client = chromadb.Client()
## 创建集合
# collection = chroma_client.delete_collection(name="my_collection")
collection = chroma_client.create_collection(name="my_collection")


In [5]:
## 向集合中添加一些文本文档
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

/slurm/home/admin/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:33<00:00, 2.52MiB/s]


In [6]:
## 查询集合
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)


{'ids': [['id1', 'id2']], 'distances': [[1.0404009819030762, 1.2430799007415771]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


## 启动持久 Chroma 客户端

In [7]:
client = chromadb.PersistentClient(path="/slurm/home/admin/nlp/DL/tests/chroma_vextor")




In [8]:
client.heartbeat() # client是否连接


1721022626040168606

In [9]:
client.reset() # 清空数据，不可逆 ALLOW_RESET=TRUE` in your environment variables

ValueError: Resetting is not allowed by this configuration (to enable it, set `allow_reset` to `True` in your Settings() or include `ALLOW_RESET=TRUE` in your environment variables)

## 在客户端/服务器模式下运行Chroma

Chroma 还可配置为以客户端/服务器模式运行。在此模式下，Chroma 客户端连接到在单独进程中运行的 Chroma 服务器。

要启动 Chroma 服务器，请运行以下命令：

In [ ]:
chroma run --path /db_path

In [10]:
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

Chroma 还提供了异步 HTTP 客户端。其行为和方法签名与同步客户端相同，但所有会阻塞的方法现在都是异步的。要使用它，请改为调用AsyncHttpClient：

In [12]:
import asyncio
import chromadb
async def main():
    client = await chromadb.AsyncHttpClient()
    collection = await client.create_collection(name="my_collection")
    await collection.add(
        documents=["hello world"],
        ids=["id1"]
    )
asyncio.run(main())

## 使用 Python HTTP 客户端
如果您在客户端-服务器模式下运行 Chroma，则可能不需要完整的 Chroma 库。相反，您可以使用轻量级的仅客户端库。 在这种情况下，您可以安装该chromadb-client软件包。此软件包是服务器的轻量级 HTTP 客户端，依赖性最小。

`pip install chromadb-client`

In [ ]:
import chromadb
# Example setup of the client to connect to your chroma server
client = chromadb.HttpClient(host='localhost', port=8000)
# Or for async usage:
async def main():
    client = await chromadb.AsyncHttpClient(host='localhost', port=8000)

## 使用集合
### 创建、检查和删除集合#
Chroma 在 URL 中使用集合名称，因此命名时有一些限制：

- 名称的长度必须介于 3 到 63 个字符之间。
- 名称必须以小写字母或数字开头和结尾，中间可以包含点、破折号和下划线。
- 名称不得包含两个连续的点。
- 该名称不能是有效的 IP 地址。

Chroma 集合使用名称和可选的嵌入函数创建。如果您提供嵌入函数，则每次获取集合时都必须提供该函数。

In [ ]:
collection = client.create_collection(name="my_collection", embedding_function=emb_fn)
collection = client.get_collection(name="my_collection", embedding_function=emb_fn)


嵌入函数以文本为输入，并执行标记化和嵌入。如果没有提供嵌入函数，Chroma 将默认使用[句子转换器](https://www.sbert.net/index.html)。
> 您可以了解有关🧬[嵌入函数的更多信息](https://docs.trychroma.com/guides/embeddings)，以及如何创建自己的函数。

可以使用 按照名称检索现有集合`.get_collection`，并使用 删除集合`.delete_collection`。您还可以使用`.get_or_create_collection`获取集合（如果存在）或创建集合（如果不存在）。


In [ ]:
collection = client.get_collection(name="test") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
collection = client.get_or_create_collection(name="test") # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
client.delete_collection(name="my_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible


集合有一些实用的便捷方法。